## Package Description

`bmdrc` is a python library for calculating **B**ench**m**ark **D**ose **R**esponse **C**urves for dichotomous (binary) and light photomotor (continuous converted to dichotomous) data. This 
tutorial will take users through the library using a light photomotor response (LPR - continuous convereted to binary response) example. `bmdrc` is broken up into five modules 


![five_modules](../bmdrc.png): 


1. *Input Data Module:* Import data into the python library

2. *Pre-Processing Module:* Combine and remove endpoints as needed

3. *Filtering Modules:* Apply the EPA recommendations for filtering 

4. *Model Fitting Modules:* Fit EPA-recommended models to data

5. *Output Modules:* Select tables to output as csvs. View plots in a HTML report.

First, install the package from pip using: 

`pip install bmdrc`

Or from github using: 

`pip install git+https://github.com/PNNL-CompBio/bmdrc`

In [1]:
# Load locally installed libraries
import bmdrc
import pandas as pd

## Module 1: Input Data Modules 

The first step to using the bmdrc python library is to properly format the data. For light photomotor response data, use the `LPRClass` object which can be imported using `from bmdrc.LPRClass import LPRClass`. Next, read the data as a pandas dataframe. 

Note that the following columns must exist and be specified in the following format: chemical identifier (chemical.id), concentration (conc), plate ID (plate.id), well name (well), time (variable, listed as "t0" for the first measurement, "t1" for the second, etc), and value (movement in mm). The data must be provided in long format, as wide format is not an acceptable format option for LPR data. 

In [2]:
pd.read_csv("../data/LPR_Long.csv").head()

,chemical.id,conc,plate.id,well,variable,value
0,2,0.0,20544,H01,t0,0.000
1,2,0.0,20544,H02,t0,NaN
2,2,0.0,20544,H03,t0,NaN
3,2,0.0,20544,H04,t0,1.083
4,2,0.0,20544,H05,t0,1.608


The following information must be provided to calculate LPR cycles: the cycle_length, the cycle_cooldown, and the starting_cycle value. In this specific instance, cycles are measured in 6 second intervals, so a length of 20 represents 120 second or 2 minutes. Between 2 minute measurement times, there are 1 minute cooldowns. Here, we calculate the difference in the area under the curve (AUC) between dark and light cycles, so specifying whether the first cycle - the starting cycle - is a "light" or "dark" cycle is important. The other endpoint that is calculated for this data, besides AUC, is MOV, which is the difference in movement between transtion periods. Here we have an example where the cycle_length is 120 seconds (20 sets of 6 second intervals), cycel_cooldown is 60 seconds (10 sets of 6 second intervals), and the first measurement is a "light" cycle.

![LPR_Image](./LPR_Example.png)

In [14]:
from bmdrc.LPRClass import LPRClass

# Convert the continuous data to dichotomous 
LPR = LPRClass(
    #df = pd.read_csv("../data/LPR_Long.csv"),
    df = pd.read_csv("~/Git_Repos/srpAnalytics/zfBmd/files/Tanguay_Phase_4_zf_104alkyl_PAH_LPR_data_PNNL_2023OCT05.csv"),
    chemical = "chemical.id",
    plate = "plate.id",
    well = "well",
    concentration = "conc",
    time = "variable",
    value = "value",
    cycle_length = 20.0,
    cycle_cooldown = 10.0, 
    starting_cycle = "light"
)

...defining cycles
...calculating AUC values
...calculating MOV values


In [15]:
# Here we can see the dichotomous data, which follows the same format as we've seen for the morphological example in the "Binary Class Example" jupyter notebook
LPR.df.head()

,chemical.id,conc,plate.id,well,endpoint,value
0,69.0,0.0,25825.0,A01,AUC1,0.0
1,69.0,0.0,25825.0,A02,AUC1,0.0
2,69.0,0.0,25825.0,A03,AUC1,1.0
3,69.0,0.0,25825.0,A04,AUC1,0.0
4,69.0,0.0,25825.0,A05,AUC1,0.0


## Module 2: Pre-Processing Modules

For a full example of all functions is available in the "Binary Class Example" notebook. Here we will cover a few functions that are relevant for LPR data.

### LPR Class: Combine Endpoints

In [5]:
LPR.combine_and_create_new_endpoints({"ANY_MOV": ["MOV1", "MOV2", "MOV3", "MOV4"],
                                      "ANY_AUC": ["AUC1", "AUC2", "AUC3", "AUC4"]})

## Module 3: Filtering Modules

For a full example of all functions is available in the "Binary Class Example" notebook. Here we will cover a few functions that are relevant for LPR data.

### Negative Control Filter

In [35]:
LPR.filter_negative_control(percentage = 50, apply = True, diagnostic_plot = True)

### Minimum Concentration Filter

In [17]:
LPR.filter_min_concentration(count = 3, apply = True, diagnostic_plot = True)

### Correlation Score Filter

In [18]:
LPR.filter_correlation_score(score = 0.2, diagnostic_plot = True, apply = True)

### Review what is kept

Following EPA recommendations of filtering, only two endpoint and chemical combinations are considered for model fitting. 

In [19]:
# See summary statistics as well as benchmark dose calculations 
LPR.plate_groups[LPR.plate_groups["bmdrc.filter"] == "Keep"]["endpoint"].unique()

array(['MOV1', 'MOV2', 'MOV4', 'AUC4', 'MOV3', 'AUC2', 'AUC3', 'AUC1'],
      dtype=object)

## Module 4: Fit Models

All 7 EPA recommended models, including one additional model (Quantal Linear) are fit to the curve. The best fit is then selected in this order:

1. If the goodness-of-fit (GOF) is over the threshold (default of 0.1)

2. The lowest Akaike Information Criterion (AIC) within 2

3. If necessary, the lowest BMDL value 

The 7 recommended models are: Logistic, Log-Logistic, Probit, Log-Probit, Weibull, Gamma, and Multistage 2

In [20]:
# Set the model fits to the recommended GOF and AIC thresholds. Any models within the AIC threshold (within 2 of the lowest score, by default), are then decided by the model_selection method. 
# Currently, only "lowest BMDL" is supported. 
LPR.fit_models(gof_threshold = 0.1, aic_threshold = 2, model_selection = "lowest BMDL")

In [37]:
LPR.plate_groups[LPR.plate_groups["bmdrc.filter"] == "Remove"]

,chemical.id,conc,plate.id,endpoint,bmdrc.num.tot,bmdrc.num.nonna,bmdrc.num.affected,bmdrc.Plate.ID,bmdrc.Endpoint.ID,bmdrc.filter,bmdrc.filter.reason


In [30]:
LPR.make_plate_groups()

In [33]:
LPR.plate_groups[LPR.plate_groups["bmdrc.Endpoint.ID"] == "53.0 AUC1"]

,chemical.id,conc,plate.id,endpoint,bmdrc.num.tot,bmdrc.num.nonna,bmdrc.num.affected,bmdrc.Plate.ID,bmdrc.Endpoint.ID,bmdrc.filter,bmdrc.filter.reason
0,53.0,0.00,27183.0,AUC1,12,0,0.0,53.0 27183.0 AUC1,53.0 AUC1,Keep,
8,53.0,0.00,27194.0,AUC1,12,0,0.0,53.0 27194.0 AUC1,53.0 AUC1,Keep,
16,53.0,2.50,27183.0,AUC1,7,0,0.0,53.0 27183.0 AUC1,53.0 AUC1,Keep,
24,53.0,2.50,27194.0,AUC1,7,0,0.0,53.0 27194.0 AUC1,53.0 AUC1,Keep,
32,53.0,3.50,27183.0,AUC1,7,0,0.0,53.0 27183.0 AUC1,53.0 AUC1,Keep,
40,53.0,3.50,27194.0,AUC1,7,0,0.0,53.0 27194.0 AUC1,53.0 AUC1,Keep,
48,53.0,4.89,27183.0,AUC1,7,0,0.0,53.0 27183.0 AUC1,53.0 AUC1,Keep,
56,53.0,4.89,27194.0,AUC1,7,0,0.0,53.0 27194.0 AUC1,53.0 AUC1,Keep,
64,53.0,6.84,27183.0,AUC1,7,0,0.0,53.0 27183.0 AUC1,53.0 AUC1,Keep,
72,53.0,6.84,27194.0,AUC1,7,0,0.0,53.0 27194.0 AUC1,53.0 AUC1,Keep,


In [12]:
# Visualize the best fitting curve
LPR.response_curve(chemical_name = 2, endpoint_name = "AUC4", model = "log probit")

In [13]:
# Visualize a curve that does not fit as well 
LPR.response_curve(chemical_name = 2, endpoint_name = "AUC4", model = "probit")

## Module 5: Output Modules

### Benchmark Dose

Build reports with the `.report()` function. See the "example_report" folder for an example. Otherwise, output a benchmark dose table with the function `.output_benchmark_dose()` and see the results with `.output_res_benchmark_dose`


In [11]:
LPR.output_benchmark_dose()
LPR.output_res_benchmark_dose

,Chemical_ID,End_Point,Model,BMD10,BMDL,BMD50,AUC,Min_Dose,Max_Dose,AUC_Norm,DataQC_Flag,BMD_Analysis_Flag,BMD10_Flag,BMD50_Flag,bmdrc.Endpoint.ID
0,4902,AUC1,NaN,NaN,NaN,NaN,27.600357,0.0,100.0,0.276004,0,0,0,0,4902 AUC1
1,4902,AUC2,NaN,NaN,NaN,NaN,22.661190,0.0,100.0,0.226612,0,0,0,0,4902 AUC2
2,4902,AUC3,NaN,NaN,NaN,NaN,28.590119,0.0,100.0,0.285901,0,0,0,0,4902 AUC3
3,4902,AUC4,NaN,NaN,NaN,NaN,18.221071,0.0,100.0,0.182211,0,0,0,0,4902 AUC4
4,4902,MOV1,NaN,NaN,NaN,NaN,16.834881,0.0,100.0,0.168349,0,0,0,0,4902 MOV1
5,4902,MOV2,NaN,NaN,NaN,NaN,17.595714,0.0,100.0,0.175957,0,0,0,0,4902 MOV2
6,4902,MOV3,NaN,NaN,NaN,NaN,29.438929,0.0,100.0,0.294389,0,0,0,0,4902 MOV3
7,4902,MOV4,NaN,NaN,NaN,NaN,15.012738,0.0,100.0,0.150127,0,0,0,0,4902 MOV4


### Dose

Similarly, see the dosage information with `.output_dose_table()` and see the actual table under `.output_res_dose_table`

In [12]:
LPR.output_dose_table()
LPR.output_res_dose_table

,Chemical_ID,End_Point,Dose,ids,num.affected,num.nonna,CI_Lo,CI_Hi
0,4902,AUC1,0.00,4902 AUC1,8.0,24,0.179722,0.532937
1,4902,AUC1,2.50,4902 AUC1,5.0,14,0.163447,0.612356
2,4902,AUC1,3.50,4902 AUC1,5.0,14,0.163447,0.612356
3,4902,AUC1,4.89,4902 AUC1,4.0,14,0.117214,0.546491
4,4902,AUC1,6.84,4902 AUC1,6.0,14,0.213808,0.674094
...,...,...,...,...,...,...,...,...
99,4902,MOV4,26.10,4902 MOV4,3.0,14,0.075714,0.475892
100,4902,MOV4,36.60,4902 MOV4,4.0,14,0.117214,0.546491
101,4902,MOV4,51.10,4902 MOV4,1.0,14,0.012722,0.314687
102,4902,MOV4,71.50,4902 MOV4,1.0,14,0.012722,0.314687


### Fits

And finally, see the fit information with `.output_fits_table()` and see the actual table under `.output_res_fits_table`

In [13]:
LPR.output_fits_table()
LPR.output_res_fits_table

,Chemical_ID,End_Point,X_vals,Y_vals
0,4902,AUC1,NaN,NaN
0,4902,AUC2,NaN,NaN
0,4902,AUC3,NaN,NaN
0,4902,AUC4,NaN,NaN
0,4902,MOV1,NaN,NaN
0,4902,MOV2,NaN,NaN
0,4902,MOV3,NaN,NaN
0,4902,MOV4,NaN,NaN
